In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import sklearn.svm as svm
from sklearn.model_selection import KFold

In [2]:
#digit intensity symmetry
train = {}
for digit in range(10):
    train[digit] = []
with open('features.train','r') as f:
    for line in f:
        line = [float(i) for i in line.split()]
        train[line[0]].append(line[1:])
for digit in range(10):
    train[digit] = np.asarray(train[digit])

In [3]:
print(train[1].shape)

(1005, 2)


In [4]:
test = {}
for digit in range(10):
    test[digit] = []
with open('features.test','r') as f:
    for line in f:
        line = [float(i) for i in line.split()]
        test[line[0]].append(line[1:])
for digit in range(10):
    test[digit] = np.asarray(test[digit])

In [5]:
print([test[i].shape for i in range(10)])

[(359, 2), (264, 2), (198, 2), (166, 2), (200, 2), (160, 2), (170, 2), (147, 2), (166, 2), (177, 2)]


In [6]:
def get_misclassified_point_idx(a,b):
    idx = []
    for i in range(len(a)):
        if a[i] != b[i]:
            idx.append(i)
    return idx

In [7]:
def soft_margin_svm(train_data,train_class,test_data,test_class,C,kernel,degree=None):
    if kernel is 'poly':
        clf = svm.SVC(C = C, kernel = kernel, degree=degree, gamma=1,coef0 = 0)
    else:
        clf = svm.SVC(C = C, kernel = kernel,gamma=1)
    clf.fit(train_data, train_class)
    train_results = clf.predict(train_data)
    idx = get_misclassified_point_idx(train_class,train_results)
    e_in = len(idx)/float(len(train_class))
    test_results = clf.predict(test_data)
    idx = get_misclassified_point_idx(test_class,test_results)
    e_out = len(idx)/float(len(test_class))
    n_support_vecs = len(clf.support_)
    
    return e_in,e_out, n_support_vecs
    

In [8]:
def create_train_test_data_one_v_all(digit,train,test):
    train_data = []
    train_class = []
    test_data = []
    test_class = []
    
    for i in range(10):
        if i == digit:
            class_id = -1
        else:
            class_id = 1

        train_data.extend(train[i])
        train_class.extend([class_id for j in train[i]])
        test_data.extend(test[i])
        test_class.extend([class_id for j in test[i]])
        
    return np.asarray(train_data),np.asarray(train_class), np.asarray(test_data), np.asarray(test_class)




In [9]:
def create_train_test_data_one_v_one(digit_1,digit_2,train,test):
    train_data = []
    train_class = []
    test_data = []
    test_class = []
    
    for i in range(10):
        if i == digit_1:
            class_id = -1
        elif i == digit_2:
            class_id = 1
        else:
            continue

        train_data.extend(train[i])
        train_class.extend([class_id for j in train[i]])
        test_data.extend(test[i])
        test_class.extend([class_id for j in test[i]])
        
    return np.asarray(train_data),np.asarray(train_class), np.asarray(test_data), np.asarray(test_class)



In [10]:
def get_classification_error(X, w, y):
    correct_idx = []
    count = 0
    for x in X:
        if np.sign(w.dot(x)) == y[count]:
            correct_idx.append(count)
        count += 1
    class_err = 1-len(correct_idx)/float(count) 
    return class_err

In [30]:
def linear_regression_weight_decay(X, y, l):
    print(l)
    X_plus = np.linalg.inv(X.transpose().dot(X) + l*np.eye(X.shape[1])).dot(X.transpose())
    w = X_plus.dot(y)
    return(w)

In [12]:
def nonlinear_transform(X, k):
    new_X = []
    for x in X:
        x1 = x[0]
        x2 = x[1]
        new_x = [1, x1, x2, x1*x2, x1**2, x2**2]
        new_x = [new_x[i] for i in range(k+1)]
        new_X.append(new_x)
    return np.asarray(new_X)

In [21]:
def no_transform(X):
    new_X = []
    for x in X:
        x1 = x[0]
        x2 = x[1]
        new_x = [1, x1, x2]
        new_X.append(new_x)
    new_X = np.asarray(new_X)
#     print(new_X.shape)
    return new_X

In [22]:
#x vs all experiments for each digit x
l = 1
e_ins = []
e_outs = []
support_vecs = []
for digit in range(10):
    print('Digit: '+str(digit))
    X, y, test_X, test_y = create_train_test_data_one_v_all(digit,train,test)
#     print(X.shape)
    X = no_transform(X)
#     print(X.shape)
    w = linear_regression_weight_decay(X,y,l)
    e_in = get_classification_error(X,w,y)
#     print(test_X.shape)
#     print(no_transform(test_X))
    test_X = no_transform(test_X)
#     print(test_X.shape)
    e_out = get_classification_error(test_X,w,test_y)
    
    e_ins.append(e_in)
    e_outs.append(e_out)

    print('E_in: '+str(e_in))
    print('E_out: '+str(e_out))

    

Digit: 0
E_in: 0.109312851461
E_out: 0.11509715994
Digit: 1
E_in: 0.0152242490742
E_out: 0.0224215246637
Digit: 2
E_in: 0.100260595254
E_out: 0.0986547085202
Digit: 3
E_in: 0.0902482512687
E_out: 0.0827105132038
Digit: 4
E_in: 0.0894253188863
E_out: 0.0996512207275
Digit: 5
E_in: 0.0762584007681
E_out: 0.079720976582
Digit: 6
E_in: 0.0910711836511
E_out: 0.0847035376183
Digit: 7
E_in: 0.0884652311068
E_out: 0.0732436472347
Digit: 8
E_in: 0.0743382252092
E_out: 0.0827105132038
Digit: 9
E_in: 0.0883280757098
E_out: 0.0881913303438


In [23]:
#x vs all experiments for each digit x, and nonlinear tform
l = 1
k = 5
e_ins = []
e_outs = []
support_vecs = []
for digit in range(10):
    print('Digit: '+str(digit))
    X, y, test_X, test_y = create_train_test_data_one_v_all(digit,train,test)
    
    Z = nonlinear_transform(X,k)
    w = linear_regression_weight_decay(Z,y,l)
    e_in = get_classification_error(Z,w,y)
    
    test_Z = nonlinear_transform(test_X,k)
    e_out = get_classification_error(test_Z,w,test_y)
    
    e_ins.append(e_in)
    e_outs.append(e_out)

    print('E_in: '+str(e_in))
    print('E_out: '+str(e_out))



Digit: 0
E_in: 0.10231792621
E_out: 0.106626806178
Digit: 1
E_in: 0.0123439857358
E_out: 0.02192326856
Digit: 2
E_in: 0.100260595254
E_out: 0.0986547085202
Digit: 3
E_in: 0.0902482512687
E_out: 0.0827105132038
Digit: 4
E_in: 0.0894253188863
E_out: 0.0996512207275
Digit: 5
E_in: 0.0762584007681
E_out: 0.0792227204783
Digit: 6
E_in: 0.0910711836511
E_out: 0.0847035376183
Digit: 7
E_in: 0.0884652311068
E_out: 0.0732436472347
Digit: 8
E_in: 0.0743382252092
E_out: 0.0827105132038
Digit: 9
E_in: 0.0883280757098
E_out: 0.0881913303438


In [26]:
#compare with and without transform
l = 1
k = 5
e_ins = []
e_outs = []
support_vecs = []
a = True
b = True
c = True
d = True
e = False
for digit in range(10):
    print('Digit: '+str(digit))
    X, y, test_X, test_y = create_train_test_data_one_v_all(digit,train,test)
    X = no_transform(X)
    w = linear_regression_weight_decay(X,y,l)
    e_in = get_classification_error(X,w,y)
    test_X = no_transform(test_X)
    e_out = get_classification_error(test_X,w,test_y)
    
    e_ins.append(e_in)
    e_outs.append(e_out)

    print('E_in: '+str(e_in))
    print('E_out: '+str(e_out))
    
    Z = nonlinear_transform(X,k)
    w = linear_regression_weight_decay(Z,y,l)
    tform_e_in = get_classification_error(Z,w,y)
    
    test_Z = nonlinear_transform(test_X,k)
    tform_e_out = get_classification_error(test_Z,w,test_y)

    print('tform_E_in: '+str(tform_e_in))
    print('tform_E_out: '+str(tform_e_out))
    
    if tform_e_out < e_out:
        if tform_e_in > e_in:
            a = False    
    if tform_e_out > 0.95*e_out:
        b = False
    if e_out != tform_e_out:
        c = False
    if e_out > 0.95*tform_e_out:
        d = False
    if digit == 5 and tform_e_out < e_out and tform_e_out > 0.95*e_out:
        e = True

print(a,b,c,d,e)


Digit: 0
E_in: 0.109312851461
E_out: 0.11509715994
tform_E_in: 0.141544369771
tform_E_out: 0.159940209268
Digit: 1
E_in: 0.0152242490742
E_out: 0.0224215246637
tform_E_in: 0.130709093403
tform_E_out: 0.126557050324
Digit: 2
E_in: 0.100260595254
E_out: 0.0986547085202
tform_E_in: 0.100260595254
tform_E_out: 0.0986547085202
Digit: 3
E_in: 0.0902482512687
E_out: 0.0827105132038
tform_E_in: 0.0902482512687
tform_E_out: 0.0827105132038
Digit: 4
E_in: 0.0894253188863
E_out: 0.0996512207275
tform_E_in: 0.0894253188863
tform_E_out: 0.0996512207275
Digit: 5
E_in: 0.0762584007681
E_out: 0.079720976582
tform_E_in: 0.0762584007681
tform_E_out: 0.079720976582
Digit: 6
E_in: 0.0910711836511
E_out: 0.0847035376183
tform_E_in: 0.0910711836511
tform_E_out: 0.0847035376183
Digit: 7
E_in: 0.0884652311068
E_out: 0.0732436472347
tform_E_in: 0.0884652311068
tform_E_out: 0.0732436472347
Digit: 8
E_in: 0.0743382252092
E_out: 0.0827105132038
tform_E_in: 0.0743382252092
tform_E_out: 0.0827105132038
Digit: 9
E_i

In [32]:
#1 vs 5 experiments
k = 5
digit_1 = 1
digit_2 = 5
e_ins = []
e_outs = []

train_data,train_class,test_data,test_class = create_train_test_data_one_v_one(digit_1,digit_2, train,test)
for l in [.01,1,100,0]:
    print('lambda: '+str(l))
    Z = nonlinear_transform(X,k)
    w = linear_regression_weight_decay(Z,y,l)
    e_in = get_classification_error(Z,w,y)
    
    test_Z = nonlinear_transform(test_X,k)
    e_out = get_classification_error(test_Z,w,test_y)
       
    print('E_in: '+str(e_in))
    print('E_out: '+str(e_out))

        
    e_ins.append(e_in)
    e_outs.append(e_out)


if e_ins[0] == e_ins[1]:
    print("b is true")
if e_outs[0] == e_outs[1]:
    print("c is true")

lambda: 0.01
0.01
E_in: 0.0883280757098
E_out: 0.0881913303438
lambda: 1
1
E_in: 0.0883280757098
E_out: 0.0881913303438
lambda: 100
100
E_in: 0.0883280757098
E_out: 0.0881913303438
lambda: 0
0
E_in: 0.0883280757098
E_out: 0.0881913303438
b is true
c is true
